In [ ]:
import sys
sys.path.append('..')

In [1]:
from new_parser import construct, match, match_exact, parser

In [2]:
from utils import SocketReader, ReaderWrapper

In [3]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDFExpand, HKDF
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import ec

In [4]:
try:
    sock.close()
except:
    pass

sock = SocketReader("www.google.com", 443, 'tcp')

sock.connect()

In [5]:
client_hello_spec = """
8==1@_selector-24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -16(...16@suites)
    -8(...8@compressions)
    -16(
        ...@extensions(
            16==0@_selector-16@ext_server_name(
                16@data(
                    8@name_type
                    -16@host_name()
                )
            )
            |16==5@_selector-16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            |16==10@_selector-16@ext_supported_curves(
                16(...16@data)
            )
            |16==11@_selector-16@ext_supported_points(
                8(...8@supported_points)
            )
            |16==35@_selector-16@ext_session_tickets()
            |16==13@_selector-16@ext_signature_algorithms(
                16(...16@data)
            )
            |16==50@_selector-16@ext_signature_algorithms_cert(
                16(...16@data)
            )
            |16==0xff01@_selector-16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            |16==16@_selector-16@ext_alpn(
                16@alpn_protocols(...8())
            )
            |16==43@_selector-16@ext_suppoerted_versions(
                8@supported_versions(...16)
            )
            |16==44@_selector-16@ext_cookie(
                16@cookie()
            )
            |16==51@_selector-16@ext_key_share(
                16@key_shares(
                    ...(
                        16@group
                        -16@data()
                    )
                )
            )
            |16==18@_selector-16@ext_sct(
                ...16@data()
            )
            |16==24@_selector-16@ext_token_binding(

            )
            |16==42@_selector-16@empty()
            |16==45@_selector-16@ext_psk_modes(
                8@psk_modes()
            )
            |16==41@_selector-16@ext_pre_shared_key(
                16@psk_identities(
                    ...(
                        16@label()
                        -32@obfuscated_ticket_age
                    )
                )
                -16@psk_binders(
                    ...8()
                )
            )
            |16@selector-16@raw_extension()
        )
    )
)"""

client_hello_packet_spec = """
8==22@_selector-16@tls_version-16@tls_handshake(%s)
"""%client_hello_spec

In [6]:
private_key = X25519PrivateKey.generate()

public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

In [7]:
transcript = hashes.Hash(hashes.SHA256())

In [8]:
client_hello = {
    'client_hello': {
        'version': 0x0303,
        'random': 0xab6618b2bf3898950c1f4820897760317c348c3b9c8288a05e436d4ba4631d9d,
        'session_id': bytes.fromhex("130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6"),
        'suites': [0x1301],
        'compressions': [0x00],
        'extensions': [
            {
                'ext_server_name': {
                    'data': {
                        'name_type': 0x00, 'host_name': "www.google.com"
                    }
                }, '_selector': 0x0000
            },
            {
                'ext_status_request': {
                    'status_type': 0x01, 'responder_id_list': 0x0000, 'request_extensions': 0x0000
                },
                '_selector': 0x0005
            },
            {
                'ext_supported_curves': {
                    'data': [0x001d]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': [0x00]
                }, '_selector': 0x000b
            },
            {
                'ext_signature_algorithms': {
                    'data': [0x0403]
                }, '_selector': 0x000d
            },
            {
                'ext_renegotiation_info': {
                    'secure_renegotitation': ''
                }, '_selector': 0xff01
            },
            {
                'ext_alpn': {
                    'alpn_protocols': ["http/1.1"]
                }, '_selector': 0x0010
            },
            {
                'ext_sct': {'data': []}, '_selector': 0x0012
            },
            {
                'ext_suppoerted_versions': {
                    'supported_versions': [0x0304, 0x0303, 0x0302, 0x0301]
                },
                '_selector': 0x002b
            },
            {
                'ext_key_share': {
                    'key_shares': [
                        {
                            'group': 0x001d, 'data': public_key
                        }
                    ]
                },
                '_selector': 0x0033
            }
        ]
    },
    '_selector': 0x01
}

client_hello_packet = {
    "tls_version": 0x0301,
    "tls_handshake": client_hello,
    "_selector": 22
}

In [9]:
client_hello_encoded = construct(client_hello_packet_spec, client_hello_packet)
client_hello_encoded

b"\x16\x03\x01\x00\xd4\x01\x00\x00\xd0\x03\x03\xabf\x18\xb2\xbf8\x98\x95\x0c\x1fH \x89w`1|4\x8c;\x9c\x82\x88\xa0^CmK\xa4c\x1d\x9d \x13\x02b\xb1\xedb=\x15\xda\xe1\x18\x90\\\xdeH\x93n\xdc\xf5x\xfaB-\x94lR\x85B=b4\xe6\x00\x02\x13\x01\x01\x00\x00\x85\x00\x00\x00\x13\x00\x11\x00\x00\x0ewww.google.com\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x04\x00\x02\x00\x1d\x00\x0b\x00\x02\x01\x00\x00\r\x00\x04\x00\x02\x04\x03\xff\x01\x00\x01\x00\x00\x10\x00\x0b\x00\t\x08http/1.1\x00\x12\x00\x00\x00+\x00\t\x08\x03\x04\x03\x03\x03\x02\x03\x01\x003\x00&\x00$\x00\x1d\x00 \xf9S\x82\xfb\xa9\x80\xa8\xed_\xaaI`\x17\x0e\xa5\xfc\xf9ou\xdb{I\x1f\x0e\xec\x8e\x02\x1cE\xc1'u"

In [10]:
transcript.update(client_hello_encoded[5:])

In [11]:
sock.send(client_hello_encoded)

In [12]:
def read_tls(sock):
    header = bytearray(sock.read_len(5))
    l = (header[3] << 8) + header[4]
    body = sock.read_len(l)
    return header + body

In [13]:
server_hello_encoded = read_tls(sock)

In [14]:
server_hello_encoded

bytearray(b'\x16\x03\x03\x00z\x02\x00\x00v\x03\x03\x7f\xf9\x91\xd1{\xdczq\xcb\x8fZ0>\x9d\x06\xa0$"vI\xcf\x81\xa3\x88\x94\xd6\xb9\x18\xeb\xec\xa5\xe4 \x13\x02b\xb1\xedb=\x15\xda\xe1\x18\x90\\\xdeH\x93n\xdc\xf5x\xfaB-\x94lR\x85B=b4\xe6\x13\x01\x00\x00.\x003\x00$\x00\x1d\x00 \xfb\x18AG=\x0c\x05\xc7\x83\x87\xc8\x1a\x862\x17?\xf9\x9a\xf8C\xf9g\x9e\xd3K\xca3\x1d\xed-\x9cM\x00+\x00\x02\x03\x04')

In [15]:
transcript.update(server_hello_encoded[5:])

In [16]:

"""8==1@_selector-24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -16(...16@suites)
    -8(...8@compressions)
    -16(
        ...@extensions(
            16==0@_selector-16@ext_server_name(
                16@data(
                    8@name_type
                    -16@host_name()
                )
            )
            |16==5@_selector-16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            |16==10@_selector-16@ext_supported_curves(
                16(...16@data)
            )
            |16==11@_selector-16@ext_supported_points(
                8(...8@supported_points)
            )
            |16==35@_selector-16@ext_session_tickets()
            |16==13@_selector-16@ext_signature_algorithms(
                16(...16@data)
            )
            |16==50@_selector-16@ext_signature_algorithms_cert(
                16(...16@data)
            )
            |16==0xff01@_selector-16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            |16==16@_selector-16@ext_alpn(
                16@alpn_protocols(...8())
            )
            |16==43@_selector-16@ext_suppoerted_versions(
                8@supported_versions(...16)
            )
            |16==44@_selector-16@ext_cookie(
                16@cookie()
            )
            |16==51@_selector-16@ext_key_share(
                16@key_shares(
                    ...(
                        16@group
                        -16@data()
                    )
                )
            )
            |16==18@_selector-16@ext_sct(
                ...16@data()
            )
            |16==24@_selector-16@ext_token_binding(

            )
            |16==42@_selector-16@empty()
            |16==45@_selector-16@ext_psk_modes(
                8@psk_modes()
            )
            |16==41@_selector-16@ext_pre_shared_key(
                16@psk_identities(
                    ...(
                        16@label()
                        -32@obfuscated_ticket_age
                    )
                )
                -16@psk_binders(
                    ...8()
                )
            )
            |16@selector-16@raw_extension()
        )
    )
)"""

server_hello_spec = """
8==2@_selector-24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16(
        ...@extensions(
            16==5@_selector-16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            |16==11@_selector-16@ext_supported_points(
                8(8@supported_points)
            )
            |16==35@_selector-16@ext_session_tickets()
            |16==0xff01@_selector-16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            |16==16@_selector-16@ext_alpn(
                16@alpn_protocols(8())
            )
            |16==18@_selector-16@ext_sct(
                16@data()
            )
            |16==43@_selector-16@ext_suppoerted_versions(
                16@supported_versions
            )
            |16==44@_selector-16@ext_cookie(
                16@cookie()
            )
            |16==51@_selector-16@ext_key_share(
                16@group
                -16@data()
            )
            |16==41@_selector-16@ext_pre_shared_key(
                16@label()
            )
            |16@selector-16@raw_extension()
        )
    )
)
"""


server_hello_packet_spec = """
 8==22@_selector-16@tls_version-16@tls_handshake(%s)
|8==20@_selector-16@tls_version-16@change_cypher()
|8==21@_selector-16@tls_version-16@alert()
|8==23@_selector-16@tls_version-16@application_data()
"""%server_hello_spec


In [17]:
server_hello = match_exact(server_hello_packet_spec, server_hello_encoded)

In [18]:
server_hello

{'_selector': 22,
 'tls_version': 0x0303,
 'tls_handshake': {'_selector': 2,
  'server_hello': {'version': 0x0303,
   'random': 0x7ff991d17bdc7a71cb8f5a303e9d06a024227649cf81a38894d6b918ebeca5e4,
   'session_id': 0x130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6,
   'suite': 0x1301,
   'compression': 0,
   'extensions': [{'_selector': 0x0033,
     'ext_key_share': {'group': 0x001d,
      'data': 0xfb1841473d0c05c78387c81a8632173ff99af843f9679ed34bca331ded2d9c4d}},
    {'_selector': 0x002b,
     'ext_suppoerted_versions': {'supported_versions': 0x0304}}]}}}

In [19]:
server_public_key = None
for extension in server_hello['tls_handshake']['server_hello']['extensions']:
    if 'ext_key_share' in extension:
        print("found public key")
        server_public_key = extension['ext_key_share']['data'].value('bytes')
        break
    else:
        raise RuntimeError("unable to find server public key")


found public key


In [20]:
shared_key = private_key.exchange(X25519PublicKey.from_public_bytes(server_public_key))
print("calculated shared key", shared_key)

calculated shared key b'[w\xa7\x05\xc8\x06%\x07\x19}?\x19u\r\xebW\x1c-qKo\xa5\x81\x18p.T\n\xd7\x8d\x83\n'


In [21]:
server_change_cipher_encoded = read_tls(sock)

In [22]:
server_change_cipher = match_exact(server_hello_packet_spec, server_change_cipher_encoded)
server_change_cipher

{'_selector': 20, 'tls_version': 0x0303, 'change_cypher': 1}

In [23]:
sock.send(server_change_cipher_encoded)

In [24]:
def get_info(label, length, context=b''):
    return b'\x00' + bytearray((length, len(b'tls13 '+label),)) + b'tls13 ' + label + bytearray((len(context),)) + context

def extract(secret, salt):
    return HKDF(hashes.SHA256(), 32, salt, b'')._extract(secret)    
    
def derive(secret, label, prev=None):
    context = get_hash(prev)
    return expand(secret, label, 32, context)
    
def expand(secret, label, l, context=b''):
    return HKDFExpand(hashes.SHA256(), l, get_info(label, l, context)).derive(secret)

def get_hash(prev):
    if prev is None:
        prev = hashes.Hash(hashes.SHA256())
    c = prev.copy()
    return c.finalize()

def finished_hash(secret, prev=None):
    finished_key = expand(secret, b'finished', 32)
    context = get_hash(prev)
    h = HMAC(finished_key, hashes.SHA256())
    h.update(context)
    return h.finalize()
    

In [25]:
early_secret = extract(b'\0'*32, None)

extractor = derive(early_secret, b"derived")

handshake_secret = extract(shared_key, extractor)

In [26]:
server_handshake_secret = derive(handshake_secret, b"s hs traffic", transcript)

server_handshake_key = expand(server_handshake_secret, b'key', 16)
server_handshake_nonce = (expand(server_handshake_secret, b'iv', 12), 0)

In [27]:
client_handshake_secret = derive(handshake_secret, b"c hs traffic", transcript)

client_handshake_key = expand(client_handshake_secret, b'key', 16)
client_handshake_nonce = (expand(client_handshake_secret, b'iv', 12), 0)

In [28]:
server_cert_encrypted_encoded = read_tls(sock)

In [29]:
server_cert_encrypted = match_exact(server_hello_packet_spec, server_cert_encrypted_encoded)
server_cert_encrypted

{'_selector': 23,
 'tls_version': 0x0303,
 'application_data': 0x17b764832173534519a73c8267e1d9cd80e0cf73b09cb97c445dafe1075dfbba4f0fe1138ebc62015e7a17e37761df511d881c3bd8717dec52d36b0abde4409b8d9037fadb12fdb166d86d9bd76304c1439a0aef6a9ecf715dcc980f94485ff0a216383576032fa6e7616416006131c958bd42464472fd9e4f25d38d3154073f764a789b151261a6b62688295f99f69eeaed3755151da54941b8adfcf50a32f9f928a6603ea775d3d26a79748c392fb7cd5fc08ffcc30a31feb5ec056115d1eac7d2ab1132131691a3b385c917a48fa41c55e5c69fc561e52f8f0948ff6a0dc70b98968d35dbee5ad19e6b94b56c51dee72232dd1b6968717a260ff75a3fefe5b965b318a36bd1855af2b979b49e22626a530341cbb41743987c89366e4106ecb23fa82bcb10d39bff52086101af21f93ffbcb9bbe72e46b0d8b7b34309f6d01db63d128e03f60b5f93189973615cc03e85ce34134ad5a4de205503449e29299fcf8be3c8cf299481af70884efe0dd04f21222df857a141c46445ac9e0288218d031091f0d82ae9cf1bd24d434db13db40905e725c8153aded2e088fd385236e57624ab0ac2444ee066fa22f43fca1873893463e2c98248679bbd2b4d27c559fac6d721ab70f9dd48c5ca983019e33474f3c4b8

In [30]:
def update_nonce(nonce):
    return (nonce[0], nonce[1]+1)

def get_nonce(nonce):
    return bytes(a ^ b for a, b in zip(nonce[0], nonce[1].to_bytes(len(nonce[0]), 'big')))

In [31]:
def decrypt_packet(packet, key, nonce):
    packet = bytes(packet)
    cipher = AESGCM(key)
    all_server_cert_packet = cipher.decrypt(get_nonce(nonce), packet[5:], packet[:5])
    server_cert_packet, server_cert_packet_suffix = all_server_cert_packet[:-1], all_server_cert_packet[-1]
    return server_cert_packet, server_cert_packet_suffix, update_nonce(nonce)

def encrypt_packet(packet, suffix, key, nonce):
    app_header_spec = "8@packet_type-16@tls_version-16@application_data_length"

    raw_packet = packet + suffix

    header_encoded = construct(app_header_spec, {
        "packet_type": 23,
        "tls_version": 0x0303,
        "application_data_length": len(raw_packet) + 16,
    })

    
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(get_nonce(nonce), raw_packet, header_encoded)
    packet = header_encoded + encrypted_app_data
    return packet, update_nonce(nonce)

In [32]:
server_cert_packet, server_cert_packet_suffix, server_handshake_nonce = decrypt_packet(server_cert_encrypted_encoded, server_handshake_key, server_handshake_nonce)
print(server_cert_packet_suffix)
server_cert_packet

22


b'\x08\x00\x00\x11\x00\x0f\x00\x10\x00\x0b\x00\t\x08http/1.1\x0b\x00\x0e\xe3\x00\x00\x0e\xdf\x00\x04[0\x82\x04W0\x82\x03?\xa0\x03\x02\x01\x02\x02\x11\x00\x8a\xach\x10\x96\x1f\xa6P\x10\xaa\x01\x93\xf9\xfd\xec\xb50\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000;1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x1e0\x1c\x06\x03U\x04\n\x13\x15Google Trust Services1\x0c0\n\x06\x03U\x04\x03\x13\x03WR20\x1e\x17\r240613163610Z\x17\r240905163609Z0\x191\x170\x15\x06\x03U\x04\x03\x13\x0ewww.google.com0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04\xb3\xc0\xf9$\xb9\x94\xf8<\xf1`\xb3NI\x83\xc6\xaa\xfc\xe0\xbc\xba\xfc\xf4\xe2:\x07x;X\xda\xab^\xdd9\xb2\xfc\xba\xaeY\x1b\xe7\n\x07\xb9\xe3\x9aDX{\xc0\xc6\xdapc\x86\xe1q\xa8\xf9J\xe5\xbes\x93\xdc\xa3\x82\x02A0\x82\x02=0\x0e\x06\x03U\x1d\x0f\x01\x01\xff\x04\x04\x03\x02\x07\x800\x13\x06\x03U\x1d%\x04\x0c0\n\x06\x08+\x06\x01\x05\x05\x07\x03\x010\x0c\x06\x03U\x1d\x13\x01\x01\xff\x04\x020\x000\x1d\x06\x03U\x1d\x0e\x04\x16\x04\x14\xa7\xd7\x83A\x

In [33]:
multiple_packets = server_cert_packet

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
encrypted_extensions_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificates_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificate_verify_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
finished_message_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

In [34]:
handshake_app_data_spec = """
    8==8@_selector-24@encrypted_extenstions(
        16@data(
            16@alpn[bytes]
            -16@data(
                16@data(
                    8@data[str]()
                )
            )
        )
    )
    |8==11@_selector-24@certificates_data(
        8@certificate_request_context-24@certificates_data(
            ...@certificates(
                24@certificate()
                -16@extensions(
                    16==5@_selector-16@status_request(
                        8@status_type_ocsp
                        -24@ocsp_staple(
                        )
                    )
                    |16==18@_selector-16@sct(
                        ...16@data()
                    )
                )
            )
        )
    )
    |8==15@_selector-24@certificate_verify(
        16@signature_algorithm-16@signature()
    )
    |8==20@_selector-24@finished_verify_data()
    |8==22@_selector-24@certificate_status(
        8@status_type
        -24@response()
    )
    |8@_selector-24@data()
"""

In [35]:
encrypted_extensions = match_exact(handshake_app_data_spec, encrypted_extensions_packet)
encrypted_extensions

{'_selector': 8,
 'encrypted_extenstions': {'data': {'alpn': b'\x00\x10',
   'data': {'data': {'data': 'http/1.1'}}}}}

In [36]:
# certificates_packet[1:][3:][1:][3:][:3], len(certificates_packet[1:][3:][1:][3:][3:3+1115])
vvv = certificates_packet[1:][3:][1:][3:][3+1115:]
# vvv[2:][:3]
vvv[2:][3:0x050f+3]
hhh = vvv[2:][0x050f+3:]
hhh[2:][3:0x0566+3]
hhh[2:][0x0566+3:]

# 0x0566
# hex(ord('f'))

b'\x00\x00'

In [37]:
server_cert = match_exact(handshake_app_data_spec, certificates_packet)
server_cert

{'_selector': 11,
 'certificates_data': {'certificate_request_context': 0,
  'certificates_data': {'certificates': [{'certificate': 0x308204573082033fa0030201020211008aac6810961fa65010aa0193f9fdecb5300d06092a864886f70d01010b0500303b310b3009060355040613025553311e301c060355040a1315476f6f676c65205472757374205365727669636573310c300a06035504031303575232301e170d3234303631333136333631305a170d3234303930353136333630395a3019311730150603550403130e7777772e676f6f676c652e636f6d3059301306072a8648ce3d020106082a8648ce3d03010703420004b3c0f924b994f83cf160b34e4983c6aafce0bcbafcf4e23a07783b58daab5edd39b2fcbaae591be70a07b9e39a44587bc0c6da706386e171a8f94ae5be7393dca38202413082023d300e0603551d0f0101ff04040302078030130603551d25040c300a06082b06010505070301300c0603551d130101ff04023000301d0603551d0e04160414a7d78341fedfb5e132c110d8f48233ac11eb072d301f0603551d23041830168014de1b1eed7915d43e3724c321bbec34396d42b230305806082b06010505070101044c304a302106082b060105050730018615687474703a2f2f6f2e706b692e676f6f672f77723230

In [38]:
import OpenSSL.crypto

first_cert = None
for cert in server_cert['certificates_data']['certificates_data']['certificates']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].value('bytes'))
    if first_cert is None:
        first_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            8a:ac:68:10:96:1f:a6:50:10:aa:01:93:f9:fd:ec:b5
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Google Trust Services, CN=WR2
        Validity
            Not Before: Jun 13 16:36:10 2024 GMT
            Not After : Sep  5 16:36:09 2024 GMT
        Subject: CN=www.google.com
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:b3:c0:f9:24:b9:94:f8:3c:f1:60:b3:4e:49:83:
                    c6:aa:fc:e0:bc:ba:fc:f4:e2:3a:07:78:3b:58:da:
                    ab:5e:dd:39:b2:fc:ba:ae:59:1b:e7:0a:07:b9:e3:
                    9a:44:58:7b:c0:c6:da:70:63:86:e1:71:a8:f9:4a:
                    e5:be:73:93:dc
                ASN1 OID: prime256v1
                NIST CURVE: P-256
        X509v3 extensions:
            X509v3 Key Usage: critical
                Digital Signature
 

In [39]:
cert_pub = first_cert.get_pubkey().to_cryptography_key()

In [40]:
transcript.update(encrypted_extensions_packet)
transcript.update(certificates_packet)

In [41]:
cert_verify_message = match_exact(handshake_app_data_spec, certificate_verify_packet)
cert_verify_message

{'_selector': 15,
 'certificate_verify': {'signature_algorithm': 0x0403,
  'signature': 0x304502204e02a727791eb5b8af7b0abda07855f3d5874323d3f8ebdc955f59db663a89f402210097aea1a3a3795215a29e57dddac92e6e92937db3343d355e94a6d14af79af016}}

In [42]:
sign_data = b'\x20' * 64 + \
    b'TLS 1.3, server CertificateVerify\x00' + \
    get_hash(transcript)

signature = cert_verify_message['certificate_verify']['signature'].value('bytes')

cert_pub.verify(signature, sign_data, ec.ECDSA(hashes.SHA256()))
print("certificate verify succeeded")

certificate verify succeeded


In [43]:
transcript.update(certificate_verify_packet)

In [44]:
finished_message = match_exact(handshake_app_data_spec, finished_message_packet)
finished_message

{'_selector': 20,
 'finished_verify_data': 0xf4af5f0fad07075b7b5d5f722aebfea6ccf41ab429d9fffcd4094000d67054ae}

In [45]:
if finished_hash(server_handshake_secret, transcript) == finished_message['finished_verify_data'].value('bytes'):
    print("matching finished hash")
else:
    raise RuntimeError("not matching finished hash")

matching finished hash


In [46]:
transcript.update(finished_message_packet)

In [47]:
client_finished_hash = finished_hash(client_handshake_secret, transcript)
client_verify_unencrypted = construct(handshake_app_data_spec, {'finished_verify_data': client_finished_hash, '_selector': 0x14})

In [48]:
client_verify_packet, client_handshake_nonce = encrypt_packet(
    client_verify_unencrypted, b'\x16', client_handshake_key, client_handshake_nonce
)

In [49]:
sock.send(client_verify_packet)

In [50]:
master_secret = extract(b'\0' * 32, derive(handshake_secret, b"derived"))
server_app_secret = derive(master_secret, b"s ap traffic", transcript)
server_app_key = expand(server_app_secret, b'key', 16)
server_app_nonce = (expand(server_app_secret, b'iv', 12), 0)
client_app_secret = derive(master_secret, b"c ap traffic", transcript)
client_app_key = expand(client_app_secret, b'key', 16)
client_app_nonce = (expand(client_app_secret, b'iv', 12), 0)

In [51]:
transcript.update(client_verify_packet)

In [52]:
http_req = b"""GET / HTTP/1.1
Host: www.google.com

"""
packet, client_app_nonce = encrypt_packet(
    http_req, b'\x17', client_app_key, client_app_nonce
)

In [53]:
sock.send(packet)

In [54]:
http_resp = read_tls(sock)

In [55]:
data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
(data, packet_type, server_app_nonce)

(b'HTTP/1.1 200 OK\r\nDate: Sun, 07 Jul 2024 20:35:34 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nContent-Security-Policy-Report-Only: object-src \'none\';base-uri \'self\';script-src \'nonce-8J4GA4QOXqRLGtwa1I_73Q\' \'strict-dynamic\' \'report-sample\' \'unsafe-eval\' \'unsafe-inline\' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: AEC=AVYB7cq_fArob1QqwxgOjBclfKy-NI8-RVQmDc-kiHT58HUmtZIJ0tY45mg; expires=Fri, 03-Jan-2025 20:35:34 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax\r\nSet-Cookie: __Secure-ENID=20.SE=kKRkyQqeR9CCnFuaaLJm_5Kh5eaIpOjXp833foTg0-VTDgLS-o3je4KctUDMDaZhc6xpUflgk3YRWs7s9WSFE7a8iPgtvb1OTNsi2pZr42ZYxp56J03-q3tvjZe0dw07-DON5BHYBx7YUmH88PN6aH33MxS1MvW9UOg_2yIlIBFJF7sXh8_i0ZAnvrE8FaprVhjFdVUarRFNhJq4D9Of; expires=Thu, 07

In [56]:
class HttpsReader(ReaderWrapper):
    def _read(self):
        global server_app_nonce
        http_resp = read_tls(sock)
        data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
        print("got body", data[:100])
        return data

In [57]:
http_reader = HttpsReader(initial_value=data)

In [58]:
http_reader.read_line()

b'HTTP/1.1 200 OK\r\n'

In [59]:
content_length = None
chunked = False

while True:
    header = http_reader.read_line()
    print(header)
    if header == b'\r\n':
        break
    if header.lower().startswith(b'content-length:'):
        content_length = int(i.split(':').strip())
        print('got content length', content_length)
    elif header.lower().startswith(b'transfer-encoding: chunked'):
        chunked = True
        print('got chunked')


b'Date: Sun, 07 Jul 2024 20:35:34 GMT\r\n'
b'Expires: -1\r\n'
b'Cache-Control: private, max-age=0\r\n'
b'Content-Type: text/html; charset=ISO-8859-1\r\n'
b"Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-8J4GA4QOXqRLGtwa1I_73Q' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp\r\n"
b'P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\n'
b'Server: gws\r\n'
b'X-XSS-Protection: 0\r\n'
b'X-Frame-Options: SAMEORIGIN\r\n'
b'Set-Cookie: AEC=AVYB7cq_fArob1QqwxgOjBclfKy-NI8-RVQmDc-kiHT58HUmtZIJ0tY45mg; expires=Fri, 03-Jan-2025 20:35:34 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax\r\n'
b'Set-Cookie: __Secure-ENID=20.SE=kKRkyQqeR9CCnFuaaLJm_5Kh5eaIpOjXp833foTg0-VTDgLS-o3je4KctUDMDaZhc6xpUflgk3YRWs7s9WSFE7a8iPgtvb1OTNsi2pZr42ZYxp56J03-q3tvjZe0dw07-DON5BHYBx7YUmH88PN6aH33MxS1MvW9UOg_2yIlIBFJF7sXh8_i0ZAnvrE8FaprVhjFdVUarRFNhJq4D9Of; expires=T

In [60]:
body = b''

In [61]:
while chunked:
    print("reading chunks")
    length = http_reader.read_line()
    print("length line", length)
    l = int(length[:-2], 16)

    ch = http_reader.read_len(l)
    print("got chunk", len(ch), ch[:100])
    body += ch
    http_reader.read_line()
    if l == 0:
        break

reading chunks
length line b'4bfa\r\n'
got body b'/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="8J4GA4QOXqRL'
got body b',248,846,1031,139,218,1283,96,225,191,311,23,1040,9,907,667,377,1,8,1,1,180,1528,208,1,1,2,93,308,44'
got body b'og=function(a,b,c,d,k,e){e=e===void 0?l:e;c||(c=t(a,b,e,d,k));if(c=r(c)){a=new Image;var g=n.length;'
got body b'z-index:1000}#gbz{left:0;padding-left:4px}#gbg{right:0;padding-right:5px}#gbs{background:transparent'
got body b't:-5px;*right:5px;*bottom:4px;-ms-filter:"progid:DXImageTransform.Microsoft.Blur(pixelradius=5)";opa'
got body b'!important}.gbtb2{display:block;border-top:2px solid transparent}.gbto .gbzt .gbtb2,.gbto .gbgt .gbt'
got body b'{padding:5px !important}.gbto #gbgs5{padding:7px 5px 6px !important}#gbi5{background:url(https://ssl'
got body b'.gbsbic::-webkit-scrollbar-track:vertical{background-color:#f5f5f5;margin-top:2px}#gbmpdv{background'
got body b'ld;white-space:nowrap}#gbmpal{*border-collapse

In [62]:
from IPython.core.display import display, HTML
display(HTML(body.decode("latin1")))

/tmp/ipykernel_2444675/449586190.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,,Geavanceerd zoeken


In [63]:
sock.close()